In [7]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np


# Read the dataset where labels is 'DIED'

In [0]:
def read_data_for_tree():
    df = pd.read_csv('dataset/covid_x_died.csv')
    label_df = pd.read_csv('dataset/covid_y_died.csv')
    encoder = LabelEncoder()
    label_df = encoder.fit_transform(label_df.values.ravel())
  
    return df, label_df

X, Y = read_data_for_tree()


# Run Decision Tree Classifier

In [5]:
rand_under = RandomUnderSampler(random_state=42)
x_resampled, y_resampled = rand_under.fit_resample(X,Y)

X_train, X_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)

print("Train_x :", X_train.shape)
print("Test_x :", X_test.shape)
print("Train_y :", y_train.shape)
print("Test_y :", y_test.shape)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_train)

# calculate the accuracy of the model on training data
accuracy = accuracy_score(y_train, y_pred)
accuracy

Train_x : (123107, 36)
Test_x : (30777, 36)
Train_y : (123107,)
Test_y : (30777,)


0.961423802058372

# Run Random Forest Classifier

In [6]:
def train_local_model(X_train, y_train):
    clf = RandomForestClassifier(max_depth=2, random_state=42)
    clf.fit(X_train, y_train)
    return clf

def aggregate_models(clients_models, X_test):
    # Collect predictions from each model
    predictions = [model.predict(X_test) for model in clients_models]

    # Aggregate predictions - here, we'll use a simple majority vote
    aggregated_prediction = np.array(predictions).mean(axis=0)
    aggregated_prediction = aggregated_prediction.round()  # Round to get the final class predictions

    return aggregated_prediction

# Example: Suppose you have 3 clients
clients_data = [(X_train, y_train), (X_train, y_train), (X_train, y_train)]

# Train local models
clients_models = [train_local_model(X, y) for X, y in clients_data]

# Aggregate models
final_prediction = aggregate_models(clients_models, X_test)

# Evaluate the aggregated model
accuracy = accuracy_score(y_test, final_prediction)

accuracy

0.9055463495467394